In [1]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [39]:
def receive_sequences(path):
    """
    
    path: a pathway to fasta file 
    
    return: a list of sequences from fasta file
    
    """
    sequences = list(SeqIO.parse(path, 'fasta'))
    
    return sequences

In [75]:
def match(new_seqs, output, threshold = 6):
    '''
    
    new_seqs: the list of input sequences
    ouput: name of output file 
    
    return : the list with alignment sequences 
    '''
    threshold_iter = len(new_seqs)*10
    score = {0: 100}
    iteration = 0
    
    while (len(new_seqs) != 1) and (max(score.values()) >= threshold) and iteration < threshold_iter:
        iteration += 1
        score = {}
        a = new_seqs[0].seq
        for j in range(1, len(new_seqs)):
            b = new_seqs[j].seq
            aln = pairwise2.align.localms(a, b, 1, -1, -1, -1)
            if aln:
                aln = aln[0]
                score[j] = aln[2]
            else:
                score[j] = 0
                      
        for k in sorted(score, key=score.get, reverse=True):
            if score[k] >= threshold:
                pair_pos = k
                c = new_seqs[pair_pos].seq
                aln_true = pairwise2.align.localms(a, c, 1, -1, -1, -1)[0]
                fragment = aln_true[0][aln_true[3]:aln_true[4]]
                flag_len = aln_true[4] - aln_true[3]
                if (a.endswith(fragment) and c.startswith(fragment)):
                    new_seqs.pop(pair_pos)
                    new_seqs.pop(0)
                    new_seqs.append(SeqRecord(a + c[flag_len:]))
                    break
                elif (c.endswith(fragment) and a.startswith(fragment)):
                    new_seqs.pop(pair_pos)
                    new_seqs.pop(0)
                    new_seqs.append(SeqRecord(c + a[flag_len:]))
                    break
            else:
                temp = new_seqs[0]
                new_seqs.pop(0)
                new_seqs.append(temp)
    
    SeqIO.write(new_seqs, fasta_output, 'fasta')

In [76]:
path = 'for_assembler_comparison_ATGTAGCTCCATTGTGTTGCCATCTCTGCACTACGCTC.fasta'

In [77]:
sequences = receive_sequences(path)

In [ ]:
match(sequences, 'out_file.fasta', threshold = 6)